In [2]:
import pandas as pd
%load_ext autoreload
%autoreload 2

%matplotlib inline

%cd ../..

!hostname

/p/fastdata/pli/Private/oberstrass1/datasets/vervet1818-3d
jrlogin04.jureca


In [3]:
import h5py as h5
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
def load_volume(clustering, method):
    path = f"data/aa/volume/{clustering}/{method}/{clustering}.h5"
    with h5.File(path, 'r') as f:
        volume = f['volume'][:]
    return path, volume

def iou_cluster(volume, cluster_ix, ignore_ix=0):
    vol_rolled = np.roll(volume, 1, axis=1)
    vol_clip = volume[:, 1:, :]
    vol_rolled_clip = vol_rolled[:, 1:, :]

    hits1 = vol_rolled_clip[vol_clip != ignore_ix] == cluster_ix
    hits2 = vol_clip[vol_clip != ignore_ix] == cluster_ix

    intersection = np.sum(hits1 & hits2)
    union = np.sum(hits1 | hits2)

    return intersection / union

In [24]:
methods = [
    "resnet18_same",
    "resnet34_circle_large",
    "resnet50_planes8_overlap",
    "pli_sobel_histo",
    "resnet18_sphere",
    "resnet34_neighbor",
    "resnet50_planes8_same",
    "pli_transret_histo",
    "resnet18_sphere_large",
    "resnet34_same",
    "resnet50_planes8_sphere",
    "resnet18_circle",
    "resnet18_sphere_small",
    "resnet34_sphere",
    "resnet50_planes8_circle",
    "resnet50_planes8_sphere_large",
    "resnet18_circle_large",
    "resnet18_touching",
    "resnet34_sphere_large",
    "resnet50_planes8_circle_large",
    "resnet50_planes8_sphere_small",
    "resnet18_neighbor",
    "resnet34_circle",
    "resnet34_touching",
    "resnet50_planes8_neighbor",
    "resnet50_planes8_touching"
]
clusterings = [
    "agglomerative_2",
    "agglomerative_5",
    "agglomerative_9",
    "agglomerative_21"
]

###

iou_frame = pd.DataFrame()

for m in methods:
    for c in clusterings:
        print(m, c)

        path, volume = load_volume(c, m)
        print(path)

        cluster_ix = np.unique(volume)[1:]

        ious = []
        for c_ix in cluster_ix:
            ious.append(iou_cluster(volume, c_ix))

        mean_iou = np.mean(ious)

        print("Mean IoU:", mean_iou)

        iou_frame = iou_frame.append({'method': m, 'clustering': c, 'iou_list': ious, 'mean_iou': mean_iou}, ignore_index=True)

iou_frame = iou_frame.sort_values(['clustering', 'method']).reset_index(drop=True)

resnet18_same agglomerative_2
data/aa/volume/agglomerative_2/resnet18_same/agglomerative_2.h5
Mean IoU: 0.9469992179529915
resnet18_same agglomerative_5
data/aa/volume/agglomerative_5/resnet18_same/agglomerative_5.h5
Mean IoU: 0.6584322339458561
resnet18_same agglomerative_9
data/aa/volume/agglomerative_9/resnet18_same/agglomerative_9.h5
Mean IoU: 0.5218136984242159
resnet18_same agglomerative_21
data/aa/volume/agglomerative_21/resnet18_same/agglomerative_21.h5
Mean IoU: 0.3978327430152493
resnet34_circle_large agglomerative_2
data/aa/volume/agglomerative_2/resnet34_circle_large/agglomerative_2.h5
Mean IoU: 0.8606614740992555
resnet34_circle_large agglomerative_5
data/aa/volume/agglomerative_5/resnet34_circle_large/agglomerative_5.h5
Mean IoU: 0.63549153887669
resnet34_circle_large agglomerative_9
data/aa/volume/agglomerative_9/resnet34_circle_large/agglomerative_9.h5
Mean IoU: 0.5356609932916234
resnet34_circle_large agglomerative_21
data/aa/volume/agglomerative_21/resnet34_circle_lar

In [8]:
models = ['resnet50_planes8', 'resnet18', 'resnet34', 'resnet50_planes64', 'pli_histo', 'pli_sobel_histo', 'pli_transret_histo']

def method_split(method):
    model = None
    for m in models:
        if method.startswith(m):
            model = m
    context = method.split(model)[-1]
    if len(context) > 0 and context[0] == "_":
        context = context[1:]
    return model, context

for k, r in iou_frame.iterrows():
    m, c = method_split(r['method'])
    iou_frame.loc[k, 'method'] = m
    iou_frame.loc[k, 'context'] = c

In [10]:
# iou_frame = pd.read_json("data/aa/volume/cluster_consistency.json", orient='index')
iou_frame.head()

,method,clustering,iou_list,mean_iou,context
0,pli_sobel_histo,agglomerative_2,"[0.93706219, 0.9637444151000001]",0.950403,
1,pli_transret_histo,agglomerative_2,"[0.9879959152000001, 0.2127986348]",0.600397,
2,resnet18,agglomerative_2,"[0.4880519793, 0.8039395008]",0.645996,circle
3,resnet18,agglomerative_2,"[0.9438757914, 0.3737190105]",0.658797,circle_large
4,resnet18,agglomerative_2,"[0.9728140311000001, 0.6711960574]",0.822005,neighbor


In [9]:
import json

#with open("data/aa/volume/cluster_consistency.json", 'w') as f:
#    out = iou_frame.to_json(orient='index')
#    out = json.dumps(json.loads(out), indent=1)
#    f.write(out)

In [68]:
iou_frame.loc[iou_frame['clustering'] == 'agglomerative_2'].sort_values('mean_iou')

,method,clustering,iou_list,mean_iou,context
18,resnet50_planes8,agglomerative_2,"[0.9312133586, 0.2301058483]",0.580660,circle_large
1,pli_transret_histo,agglomerative_2,"[0.9879959152000001, 0.2127986348]",0.600397,
2,resnet18,agglomerative_2,"[0.4880519793, 0.8039395008]",0.645996,circle
3,resnet18,agglomerative_2,"[0.9438757914, 0.3737190105]",0.658797,circle_large
6,resnet18,agglomerative_2,"[0.9629396558000001, 0.6755886599]",0.819264,sphere
4,resnet18,agglomerative_2,"[0.9728140311000001, 0.6711960574]",0.822005,neighbor
8,resnet18,agglomerative_2,"[0.9419461183000001, 0.7078085632000001]",0.824877,sphere_small
21,resnet50_planes8,agglomerative_2,"[0.8627587116000001, 0.7911644714]",0.826962,same
11,resnet34,agglomerative_2,"[0.9399713699000001, 0.7813515783]",0.860661,circle_large
7,resnet18,agglomerative_2,"[0.9108827006, 0.8834761234]",0.897179,sphere_large


In [67]:
iou_frame.loc[iou_frame['clustering'] == 'agglomerative_5'].sort_values('mean_iou')

,method,clustering,iou_list,mean_iou,context
53,pli_transret_histo,agglomerative_5,"[0.9635357497, 0.8719239120000001, 0.640690983...",0.537436,
73,resnet50_planes8,agglomerative_5,"[0.6073738360000001, 0.8443562228, 0.512564654...",0.558607,same
70,resnet50_planes8,agglomerative_5,"[0.9161549691, 0.5417567276, 0.6492183861, 0.2...",0.599652,circle_large
69,resnet50_planes8,agglomerative_5,"[0.851996853, 0.8535341150000001, 0.5421733816...",0.608637,circle
68,resnet34,agglomerative_5,"[0.6362184183, 0.4048054172, 0.806297707, 0.59...",0.608792,touching
55,resnet18,agglomerative_5,"[0.8685746591, 0.3737190105, 0.8403974891, 0.4...",0.613474,circle_large
62,resnet34,agglomerative_5,"[0.8834041254, 0.7143734935, 0.4924813818, 0.3...",0.617142,circle
61,resnet18,agglomerative_5,"[0.7021459818, 0.633234486, 0.3455369664000000...",0.621009,touching
54,resnet18,agglomerative_5,"[0.4430342152, 0.5255414485000001, 0.600510837...",0.625126,circle
63,resnet34,agglomerative_5,"[0.8815252961000001, 0.6468992905000001, 0.545...",0.635492,circle_large


In [66]:
iou_frame.loc[iou_frame['clustering'] == 'agglomerative_9'].sort_values('mean_iou')

,method,clustering,iou_list,mean_iou,context
94,resnet34,agglomerative_9,"[0.806297707, 0.5512439701, 0.3498967702000000...",0.493033,touching
88,resnet34,agglomerative_9,"[0.7814332375, 0.3030131576, 0.4924813818, 0.6...",0.495691,circle
79,pli_transret_histo,agglomerative_9,"[0.539692238, 0.8670959893, 0.6014515794, 0.21...",0.495966,
81,resnet18,agglomerative_9,"[0.7209653132, 0.3737190105, 0.5886529095, 0.2...",0.501156,circle_large
87,resnet18,agglomerative_9,"[0.5622276603, 0.5105910912, 0.345536966400000...",0.507186,touching
99,resnet50_planes8,agglomerative_9,"[0.5125646543, 0.4230168654, 0.5214479181, 0.4...",0.521556,same
83,resnet18,agglomerative_9,"[0.648385866, 0.8016397624, 0.4502615701, 0.30...",0.521814,same
103,resnet50_planes8,agglomerative_9,"[0.3523642729, 0.5761258016, 0.3126878322, 0.6...",0.525912,touching
96,resnet50_planes8,agglomerative_9,"[0.2301058483, 0.48911406500000004, 0.87880795...",0.526844,circle_large
78,pli_sobel_histo,agglomerative_9,"[0.38188271090000003, 0.430848074, 0.359626105...",0.530773,


In [65]:
iou_frame.loc[iou_frame['clustering'] == 'agglomerative_21'].sort_values('mean_iou')

,method,clustering,iou_list,mean_iou,context
47,resnet50_planes8,agglomerative_21,"[0.6382377804, 0.453656769, 0.2742922719, 0.32...",0.347820,same
26,pli_sobel_histo,agglomerative_21,"[0.2834783491, 0.4614515878, 0.1572511369, 0.5...",0.364755,
51,resnet50_planes8,agglomerative_21,"[0.193308794, 0.165374882, 0.4241729527, 0.190...",0.372490,touching
27,pli_transret_histo,agglomerative_21,"[0.48764624500000003, 0.2274298105, 0.34505417...",0.372842,
44,resnet50_planes8,agglomerative_21,"[0.1562598333, 0.1404481196, 0.3100547336, 0.4...",0.377047,circle_large
37,resnet34,agglomerative_21,"[0.5206857017000001, 0.7182008667, 0.366033400...",0.378287,circle_large
39,resnet34,agglomerative_21,"[0.1969130393, 0.1716525214, 0.3745601549, 0.5...",0.396541,same
31,resnet18,agglomerative_21,"[0.2594767283, 0.2711966849, 0.1935679132, 0.4...",0.397833,same
42,resnet34,agglomerative_21,"[0.3030122693, 0.2830569784, 0.3120433393, 0.5...",0.399896,touching
28,resnet18,agglomerative_21,"[0.5818374354, 0.2908329887, 0.608603374000000...",0.406449,circle


In [25]:
frame = iou_frame.loc[iou_frame['clustering'] == 'agglomerative_21'].sort_values('mean_iou')

print(frame.loc[frame['method'] == 'pli_sobel_histo'][['mean_iou', 'context']])

    mean_iou context
26  0.364755        
